### Ingest qualifying.json file

##### Step 1 - Read the JSON file using the spark dataframe reader API

In [ ]:
v_data_source = "" #Provided by Data Factory

In [ ]:
%run configuration

In [ ]:
%run common_functions

In [10]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import lit

StatementMeta(, 88cf0399-771a-46ae-a4f2-c9f0202f10c4, 12, Finished, Available)

In [11]:
qualifying_schema = StructType(fields=[StructField("qualifyId", IntegerType(), False),
                                      StructField("raceId", IntegerType(), True),
                                      StructField("driverId", IntegerType(), True),
                                      StructField("constructorId", IntegerType(), True),
                                      StructField("number", IntegerType(), True),
                                      StructField("position", IntegerType(), True),
                                      StructField("q1", StringType(), True),
                                      StructField("q2", StringType(), True),
                                      StructField("q3", StringType(), True),
                                     ])

StatementMeta(, 88cf0399-771a-46ae-a4f2-c9f0202f10c4, 13, Finished, Available)

In [12]:
qualifying_df = spark.read \
.schema(qualifying_schema) \
.option("multiLine", True) \
.json(f"{bronze_folder_path}/qualifying")
display(qualifying_df)

StatementMeta(, 88cf0399-771a-46ae-a4f2-c9f0202f10c4, 14, Finished, Available)

SynapseWidget(Synapse.DataFrame, ad55acb9-f8d2-4e61-be7c-238f3f1bfc28)

##### Step 2 - Rename columns and add new columns
1. Rename qualifyingId, driverId, constructorId and raceId
1. Add ingestion_date with current timestamp

In [ ]:
final_df = add_ingestion_date(qualifying_df)

In [14]:
final_df = final_df.withColumnRenamed("qualifyId", "qualify_id") \
.withColumnRenamed("driverId", "driver_id") \
.withColumnRenamed("raceId", "race_id") \
.withColumnRenamed("constructorId", "constructor_id") \
.withColumn("data_source", lit(v_data_source))
display(final_df)

StatementMeta(, 88cf0399-771a-46ae-a4f2-c9f0202f10c4, 16, Finished, Available)

SynapseWidget(Synapse.DataFrame, 2d97d440-9232-495c-9eb4-d0c390a59f1f)

##### Step 3 - Write to output to processed container in parquet format

In [15]:
final_df.write.mode('overwrite').parquet(f"{silver_folder_path}/qualifying")

StatementMeta(, 88cf0399-771a-46ae-a4f2-c9f0202f10c4, 17, Finished, Available)

In [16]:
df_parquet = spark.read.parquet(f"{silver_folder_path}/qualifying")
display(df_parquet)

StatementMeta(, 88cf0399-771a-46ae-a4f2-c9f0202f10c4, 18, Finished, Available)

SynapseWidget(Synapse.DataFrame, ca15b263-bc93-4344-8e7a-4a8102d2cf27)